# Reinforcement Learning for Trading

Author: Elkin Narváez Paz

* Email: elkinnarvaez2406@gmail.com
* LinkedIn: https://www.linkedin.com/in/elkinnarvaez/
* GitHub: https://github.com/elkinnarvaez

© All rights reserved.

## Libraries

In [44]:
# Data gathering
import http.client
import requests
import json
import datetime

# Statistics
import statistics as stats

## Data gathering

In [49]:
symbols = ['PFE']
indicators = {}

### Pfizer Inc. (PFE)

In [50]:
conn = http.client.HTTPSConnection("yh-finance.p.rapidapi.com")

headers = {
    'x-rapidapi-host': "yh-finance.p.rapidapi.com",
    'x-rapidapi-key': "fc1708951fmshca8b1e8a9e96604p126192jsnf88ca47a0efd"
    }

conn.request("GET", "/stock/v2/get-chart?interval=60m&symbol=PFE&range=2y&region=US", headers=headers)

res = conn.getresponse()
data = res.read()

In [51]:
result = json.loads(data.decode("utf-8"))["chart"]["result"][0]

In [52]:
indicators['PFE'] = {
    "timestamp": list(map(lambda x: datetime.datetime.fromtimestamp(x), result["timestamp"])),
    "high": result["indicators"]["quote"][0]["high"],
    "volume": result["indicators"]["quote"][0]["volume"],
    "open": result["indicators"]["quote"][0]["open"],
    "low": result["indicators"]["quote"][0]["low"],
    "close": result["indicators"]["quote"][0]["close"]
}

## Creation of trading signals

In [53]:
tradingSignals = {}
for symbol in symbols:
    tradingSignals[symbol] = {}

### Trading signals functions

#### Simple Moving Average (SMA)

In [63]:
def SMA(symbol, numPeriods):
    """
        Input:
            - symbol: the symbol of the stock for which the SMA will be calculated to -> str
            - numPeriods: number of time periods over which to average -> int
        Output:
            SMA values
    """
    smaValues = []
    
    history = [] # history of prices
    for closePrice in indicators[symbol]["close"]:
        history.append(closePrice)
        if(len(history) > numPeriods): # remove the oldest price because we only average over the last 'timePeriod' prices
            del history[0]
        smaValues.append(stats.mean(history))
        
    return smaValues

#### Exponential Moving Average (EMA)

In [64]:
def EMA(symbol, numPeriods):
    """
        Input:
            - symbol: the symbol of the stock for which the SMA will be calculated to -> str
            - numPeriods: number of time periods over which to average -> int
        Output:
            EMA values
    """
    emaValues = []
    
    K = 2 / (numPeriods + 1) # smoothing constant
    ema = 0
    
    for closePrice in indicators[symbols]["close"]:
        if(ema == 0):
            ema = closePrice
        else:
            ema = (closePrice - ema) * K + ema
        emaValues.append(ema)
    
    return emaValues

#### Absolute Price Oscillator (APO)

#### Moving Average Convergence Divergence (MACD)

#### Bollinger bands (BBANDS)

### Pfizer Inc. (PFE)